In [ ]:
# Distance modulus of Type Ia supernovae (arXiv:1710.00845)
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

z = [0.17331, 0.21946, 0.26862, 0.2836 , 0.30158, \
     0.31156, 0.32454, 0.34052, 0.3507 , 0.36791, \
     0.37386, 0.40455, 0.43544, 0.46156, 0.47572, \
     0.48881, 0.51961, 0.53532, 0.58375, 0.62063, \
     0.68196, 0.74   , 0.78904, 0.81773, 0.85073, \
     0.92624, 0.99781, 1.12   , 1.23   , 1.33   ]
mB = [20.15345, 20.8469 , 21.1762 , 21.40915, 21.64245, \
      21.7417 , 21.79365, 21.9463 , 21.9623 , 22.24455, \
      22.3047 , 22.44625, 22.36675, 22.76265, 22.6952 , \
      22.8653 , 23.03865, 23.06945, 23.2686 , 23.5114 , \
      23.59755, 23.8446 , 24.2188 , 24.3431 , 24.31865, \
      24.7687 , 24.6122 , 25.0639 , 25.4985 , 25.6034 ]
dmB = [0.09925, 0.1023 , 0.0993 , 0.1028 , 0.0939 , \
       0.10825, 0.10225, 0.10555, 0.10725, 0.1019 , \
       0.0854 , 0.1075 , 0.109  , 0.11115, 0.109  , \
       0.1136 , 0.109  , 0.11085, 0.09915, 0.1131 , \
       0.1484 , 0.12605, 0.11585, 0.12085, 0.1304 , \
       0.10935, 0.0953 , 0.1649 , 0.17085, 0.1484 ]


z = np.array(z)
mB = np.array(mB)
dmB = np.array(dmB)

#TODO
MB = ...
muB = ...

plt.errorbar(z, muB, yerr=dmB, color='k', ls='None')
plt.xlabel(r'$z$')
plt.ylabel(r'$\mu$')
plt.show()

In [ ]:
# Modulus in different cosmological models
from astropy.cosmology import LambdaCDM

plt.errorbar(z, muB, yerr=dmB, color='k', ls='None')

#TODO
Om = ...
OL = ...
z0 = np.linspace(0.1, 1.4, 50)
label = r'$\Omega_{{\rm m}}={0:g}, \Omega_\Lambda={1:g}$'

for i in range(len(Om)):
    cosmo = LambdaCDM(H0=70, Om0=Om[i], Ode0=OL[i])
    mu = cosmo.distmod(z0)
    plt.plot(z0, mu, label=label.format(Om[i],OL[i]))
    
plt.xlabel(r'$z$')
plt.ylabel(r'$\mu$')
plt.legend()
plt.show()

In [ ]:
# Fitting of cosmological parameters
from scipy.optimize import minimize

def chi2(par, z, muB, dmB):
    H0, Om, OL = par
    cosmo = LambdaCDM(H0, Om0=Om, Ode0=OL)
    mu = cosmo.distmod(z).value
    #TODO
    diff = ...
    return ...

ini = [70., 0.3, 0.7]
res = minimize(chi2, ini, args=(z,muB,dmB))

H0, Om, OL = res.x
print('Best-fit:')
print('  H0 = {0:g}'.format(H0))
print('  Om = {0:g}'.format(Om))
print('  OL = {0:g}'.format(OL))

cosmo = LambdaCDM(H0, Om0=Om, Ode0=OL)
mu = cosmo.distmod(z0)

plt.errorbar(z, muB, yerr=dmB, color='k', ls='None')
plt.plot(z0, mu)

plt.show()

In [ ]:
# MCMC sampling of parameters
from astropy.cosmology import FlatLambdaCDM
import emcee

def chi2_flat(par, z, muB, dmB):
    H0, Om = par
    cosmo = FlatLambdaCDM(H0, Om0=Om)
    mu = cosmo.distmod(z).value
    #TODO
    diff = ...
    return ...

def lnprior(par):
    H0, Om = par
    #TODO
    if ... and ...:
        return 0.0
    return -np.inf

def lnprob(par, z, muB, dmB):
    lp = lnprior(par)
    if not np.isfinite(lp):
        return -np.inf
    #TODO
    return ...

ndim, nwalkers = 2, 50
ini = np.array([70., 0.3])
ini = [ini + 1e-4 * np.random.randn(ndim) \
        for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, \
        lnprob, args=(z, muB, dmB))
sampler.run_mcmc(ini, 500)

ax = [None] * ndim
for i in range(ndim):
    ax[i] = plt.subplot2grid((ndim,1), (i,0))
    if i != ndim - 1:
        ax[i].set_xticklabels(())

for i in range(nwalkers):
    ax[0].plot(sampler.chain[i, :, 0], 'k-', lw=0.5)
    ax[1].plot(sampler.chain[i, :, 1], 'k-', lw=0.5)

ax[0].set_ylabel(r'$H_0$')
ax[1].set_ylabel(r'$\Omega_{\rm m}$')
ax[1].set_xlabel('Step')
plt.show()

In [ ]:
# Likelihood distribution of parameters
import corner
from IPython.display import display, Math

samples = sampler.chain[:, 100:, :].reshape((-1, ndim))
fig = corner.corner(samples, labels=[r'$H_0$', \
        r'$\Omega_{\rm m}$', r'$\Omega_\Lambda$'])
fig.show()

H0, Om = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), \
        zip(*np.percentile(samples, [16, 50, 84], axis=0)))

display(Math(r'$H_0 = {0:.1f}_{{-{1:.2f}}}^{{+{2:.2f}}}$' \
        ''.format(H0[0], H0[1], H0[2])))
display(Math(r'$\Omega_{{\rm m}} = {0:.2f}_{{-{1:.3f}}}^' \
        '{{+{2:.3f}}}$'.format(Om[0], Om[1], Om[2])))